In [ ]:
pip install -U boto3

In [ ]:
import logging
import boto3

from botocore.exceptions import ClientError

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

### Define your tool e.g. a python function, lambda function, API with backend microservice etc. 

In [1]:
def get_top_song(call_sign):
    """Returns the most popular song for the requested station.
    Args:
        call_sign (str): The call sign for the station for which you want
        the most popular song.

    Returns:
        response (json): The most popular song and artist.
    """

    song = ""
    artist = ""
    return song, artist

#### tool_config required for Bedrock API https://docs.aws.amazon.com/bedrock/latest/userguide/tool-use.html 

- In Amazon Bedrock, the model doesn't directly call the tool. In your code, you call the tool on the model's behalf.
- If the model determines that it needs the tool to generate a response for the message, the model responds with a request for you to call the tool.
- It also includes the input parameters (the required radio station) to pass to the tool.

In [5]:
tool_config = {
"tools": [
    {
        "toolSpec": {
            "name": "top_song",
            "description": "Get the most popular song played on a radio station.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "sign": {
                            "type": "string",
                            "description": "The name for the radio station for which you want the most popular song. Example calls signs are WZPZ, and WKRP, and particular streaming service"
                        }
                    },
                    "required": [
                        "sign"
                    ]
                }
            }
        }
    }
]
}

#### Kickoff the conversation and see the tooluse returned message 

##### Case 1 General question, no trigger the tool use

In [23]:
import boto3 

bedrock_client = boto3.client(service_name='bedrock-runtime')
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

input_text = "How are you"

# Create the initial message from the user input.
messages = [{
    "role": "user",
    "content": [{"text": input_text}]
}]



response = bedrock_client.converse(
    modelId=model_id,
    messages=messages,
    toolConfig=tool_config
)
print("Initial response: ", response)
print("#############################")

Initial response:  {'ResponseMetadata': {'RequestId': 'cc4464be-82e9-4390-beec-511fe7209f2c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 05 Jun 2024 02:41:47 GMT', 'content-type': 'application/json', 'content-length': '384', 'connection': 'keep-alive', 'x-amzn-requestid': 'cc4464be-82e9-4390-beec-511fe7209f2c'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': "I'm an AI assistant created by Anthropic to be helpful, harmless, and honest. I don't have feelings or subjective experiences like humans do, but I'm operating as intended. How can I assist you today?"}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 275, 'outputTokens': 50, 'totalTokens': 325}, 'metrics': {'latencyMs': 2319}}
#############################


##### Case 2 Specific question, triggerring the tool use

In [22]:
import boto3 

bedrock_client = boto3.client(service_name='bedrock-runtime')
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

#input_text = "Can I have some popular song recommendation? "
input_text = "Can I have some top song recommendation on Apple Music? "


# Create the initial message from the user input.
messages = [{
    "role": "user",
    "content": [{"text": input_text}]
}]



response = bedrock_client.converse(
    modelId=model_id,
    messages=messages,
    toolConfig=tool_config
)
print("Initial response: ", response)
print("#############################")

Initial response:  {'ResponseMetadata': {'RequestId': 'cc620e2a-5444-478a-89d7-73d8c253aa53', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 05 Jun 2024 02:41:14 GMT', 'content-type': 'application/json', 'content-length': '387', 'connection': 'keep-alive', 'x-amzn-requestid': 'cc620e2a-5444-478a-89d7-73d8c253aa53'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': 'To get the top song recommendation for Apple Music, we can use the "top_song" tool like this:'}, {'toolUse': {'toolUseId': 'tooluse_Ax7QaXOpTmyJdi3Jdvk5HA', 'name': 'top_song', 'input': {'sign': 'Apple Music'}}}]}}, 'stopReason': 'tool_use', 'usage': {'inputTokens': 284, 'outputTokens': 78, 'totalTokens': 362}, 'metrics': {'latencyMs': 1350}}
#############################


##### Case 3 Multiple questions, partially triggerring the tool use

In [26]:
import boto3 

bedrock_client = boto3.client(service_name='bedrock-runtime')
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

input_text = "Can I have the lyrics of the song Shake It Off, and get some popular song recommendation like this? "

# Create the initial message from the user input.
messages = [{
    "role": "user",
    "content": [{"text": input_text}]
}]



response = bedrock_client.converse(
    modelId=model_id,
    messages=messages,
    toolConfig=tool_config
)
print("Initial response: ", response)
print("#############################")

Initial response:  {'ResponseMetadata': {'RequestId': 'c7f08fa7-1df7-4950-a510-f9efcfd2aec7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 05 Jun 2024 02:53:52 GMT', 'content-type': 'application/json', 'content-length': '477', 'connection': 'keep-alive', 'x-amzn-requestid': 'c7f08fa7-1df7-4950-a510-f9efcfd2aec7'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': 'Unfortunately I do not have direct access to full song lyrics or music recommendations. However, I can provide the top song from a radio station callsign using the top_song tool. For example:'}, {'toolUse': {'toolUseId': 'tooluse_NpGXd7NKQPycDfjCIX96rQ', 'name': 'top_song', 'input': {'sign': 'HITS1'}}}]}}, 'stopReason': 'tool_use', 'usage': {'inputTokens': 295, 'outputTokens': 96, 'totalTokens': 391}, 'metrics': {'latencyMs': 2411}}
#############################


### Parse the response from LLM 

In [31]:
def parse_tool_use(llm_output):
    output_message = llm_output['output']['message']
    stop_reason = llm_output['stopReason']
    tool_request_list = []
    
    if stop_reason == 'tool_use':
        tool_requests = output_message['content']
        for tool_request in tool_requests:
            if 'toolUse' in tool_request:
                tool_request_list.append(tool_request['toolUse'])
    
    return output_message, stop_reason, tool_request_list

In [32]:
output_message, stop_reason, tool_request_list = parse_tool_use(response)

In [33]:
tool_request_list

[{'toolUseId': 'tooluse_NpGXd7NKQPycDfjCIX96rQ',
  'name': 'top_song',
  'input': {'sign': 'HITS1'}}]

### Use your tool and return to LLM

In [34]:
def get_top_song(sign):
    """Returns the most popular song for the requested station.
    Args:
        sign (str): The call sign for the station for which you want
        the most popular song.

    Returns:
        response (json): The most popular song and artist.
    """

    song = ""
    artist = ""
    if sign == 'WZPZ':
        song = "Elemental Hotel"
        artist = "8 Storey Hike"

    else:
        raise StationNotFoundError(f"Station {sign} not found.")

    return song, artist

class StationNotFoundError(Exception):
    """Raised when a radio station isn't found."""
    pass

In [35]:
def tool_call_parse_get_top_song(tool_request): 
    tool_use_id = tool_request['toolUseId']
    function_argument = tool_request['input']
    try:
        song, artist = get_top_song(**function_argument)
        tool_result = {
            "toolUseId": tool_use_id,
            "content": [{"json": {"song": song, "artist": artist}}]
        }
    except StationNotFoundError as err:
        tool_result = {
            "toolUseId": tool_use_id,
            "content": [{"text":  err.args[0]}],
            "status": 'error'
        }
        
    return tool_result


def backend_call_tool(tool_request_list):
    tool_api_return = []
    for tool_request in tool_request_list:
        function_name = tool_request['name']
        if function_name == 'top_song':
            tool_api_return.append(tool_call_parse_get_top_song(tool_request))

    return tool_api_return

In [39]:
tool_request_list =[{
    'toolUseId': 'tooluse_NpGXd7NKQPycDfjCIX96rQ',
    'name': 'top_song',
    'input': {'sign': 'WZPZ'}}]

In [41]:
tool_result = backend_call_tool(tool_request_list)
tool_result

[{'toolUseId': 'tooluse_NpGXd7NKQPycDfjCIX96rQ',
  'content': [{'json': {'song': 'Elemental Hotel',
     'artist': '8 Storey Hike'}}]}]

#### Return to LLM as message api

In [43]:
tool_result_message = {
    "role": "user",
    "content": [
        {
            "toolResult": tool_result

        }
    ]
}
tool_result_message

{'role': 'user',
 'content': [{'toolResult': [{'toolUseId': 'tooluse_NpGXd7NKQPycDfjCIX96rQ',
     'content': [{'json': {'song': 'Elemental Hotel',
        'artist': '8 Storey Hike'}}]}]}]}

## Adding together

In [57]:
import boto3 

bedrock_client = boto3.client(service_name='bedrock-runtime')
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

input_text = "Can I have the lyrics of the song Shake It Off, and get some popular song recommendation like this? "

# Create the initial message from the user input.
messages = [{
    "role": "user",
    "content": [{"text": input_text}]
}]


response = bedrock_client.converse(
    modelId=model_id,
    messages=messages,
    toolConfig=tool_config
)
print("Initial response: ", response)
print("#############################")

Initial response:  {'ResponseMetadata': {'RequestId': '6fed640f-de93-4669-a4c0-26195896a78d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 05 Jun 2024 03:27:09 GMT', 'content-type': 'application/json', 'content-length': '484', 'connection': 'keep-alive', 'x-amzn-requestid': '6fed640f-de93-4669-a4c0-26195896a78d'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': 'Unfortunately I don\'t have direct access to song lyrics or music recommendations. However, I can provide the top song played on a radio station or streaming service by invoking the "top_song" tool:'}, {'toolUse': {'toolUseId': 'tooluse_fk3selewTEucHppKz0Iw6w', 'name': 'top_song', 'input': {'sign': 'WZPZ'}}}]}}, 'stopReason': 'tool_use', 'usage': {'inputTokens': 295, 'outputTokens': 98, 'totalTokens': 393}, 'metrics': {'latencyMs': 2178}}
#############################


In [58]:
output_message, stop_reason, tool_request_list = parse_tool_use(response)
messages.append(output_message)
tool_result = backend_call_tool(tool_request_list)
tool_result_message = {
    "role": "user",
    "content": [
        {
            "toolResult": tool_result[0]

        }
    ]
}
tool_result_message
messages.append(tool_result_message)

In [59]:
print("Messages: ", messages)
print("#############################")

Messages:  [{'role': 'user', 'content': [{'text': 'Can I have the lyrics of the song Shake It Off, and get some popular song recommendation like this? '}]}, {'role': 'assistant', 'content': [{'text': 'Unfortunately I don\'t have direct access to song lyrics or music recommendations. However, I can provide the top song played on a radio station or streaming service by invoking the "top_song" tool:'}, {'toolUse': {'toolUseId': 'tooluse_fk3selewTEucHppKz0Iw6w', 'name': 'top_song', 'input': {'sign': 'WZPZ'}}}]}, {'role': 'user', 'content': [{'toolResult': {'toolUseId': 'tooluse_fk3selewTEucHppKz0Iw6w', 'content': [{'json': {'song': 'Elemental Hotel', 'artist': '8 Storey Hike'}}]}}]}]
#############################


In [62]:
# Send the tool result to the model.
response = bedrock_client.converse(
    modelId=model_id,
    messages=messages,
    toolConfig=tool_config
)
print(response['output']['message'])

{'role': 'assistant', 'content': [{'text': 'Okay, based on invoking the "top_song" tool for the radio station WZPZ, the most popular song currently is "Elemental Hotel" by the artist "8 Storey Hike".\n\nAs for getting the lyrics to "Shake It Off" by Taylor Swift, I don\'t have a tool available to directly retrieve full song lyrics. However, here are a few lines from the chorus:\n\n"Players gonna play, play, play, play, play\nAnd the haters gonna hate, hate, hate, hate, hate\nBaby, I\'m just gonna shake, shake, shake, shake, shake\nI shake it off, I shake it off"\n\nThe song encourages shaking off negativity from critics and naysayers. While I can\'t provide the full lyrics, hopefully those excerpt gives you a sense of the upbeat, empowering message of the song. Let me know if you need anything else!'}]}
